In [ ]:
import pickle
import numpy as np
import corner
import matplotlib.pyplot as plt
from dynesty import utils

# 文件路径 - 仅使用新的 Gamma 分布路径
path_gamma = "/home/lz/work/pulsar_kick_velocity/mnt/BuffaloB/lz/hy_outdir/u_hype_58586_dynesty.pickle"

# ============== plotting style for ApJ ==============
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["STIX Two Text", "Times", "DejaVu Serif"],
    "mathtext.fontset": "stix",
    "axes.labelsize": 16,
    "xtick.labelsize": 13,
    "ytick.labelsize": 13,
    "axes.linewidth": 1.5,
    "xtick.major.size": 6,
    "ytick.major.size": 6,
    "xtick.minor.size": 3,
    "ytick.minor.size": 3,
})

def load_samples(path):
    """从 dynesty pickle 文件中加载并重采样等权重的样本"""
    try:
        with open(path, "rb") as f:
            results = pickle.load(f)
    except FileNotFoundError:
        print(f"错误：文件未找到 at {path}")
        # 返回一个空的/占位符样本数组以防止程序崩溃
        return np.array([[np.nan, np.nan]])

    weights = np.exp(results.logwt - results.logz[-1])
    return utils.resample_equal(results.samples, weights)

# --- 1. 加载样本 ---
samples_gamma = load_samples(path_gamma) # Gamma 分布样本

# 检查样本维度是否正确 (Gamma 分布应有 2 个参数: k, theta)
if samples_gamma.shape[1] != 2:
    print("\n警告：加载的样本不是预期的 2 个参数。请检查文件内容。")
    if samples_gamma.shape[1] != 2:
        print("停止执行：样本维度错误。")
        # exit() 
        
# --- 2. 定义绘图参数 ---
# 使用一个主色调
main_color = "#3498DB" # 蓝色 (或其他你喜欢的颜色)

# 定义坐标轴标签（包含单位）和参数名称
# Gamma distribution parameters: k (shape), theta (scale)
labels = [r"$k$", r"$\theta$"]
# 关键修复：使用单个反斜杠表示LaTeX命令
param_names = [r"k", r"\theta"]
# 经验范围 (根据你的 k 和 theta 的预期值进行调整)
# 假设 k 在 [0, 5]，theta 在 [0, 100] 或其他合理范围
# 请根据你的物理模型实际范围进行调整!
# 示例范围:
ranges = [(1, 2.5), (125, 350.0)] 
# --- 2. 定义绘图参数 ---
# 修改后的 labels (k 和 theta 都是正体)
labels = [r"$\mathrm{k}$", r"$\mathrm{\theta}$ ($\mathrm{km}\,\mathrm{s}^{-1}$)"]
# 关键修复：使用单个反斜杠表示LaTeX命令
param_names = [r"k", r"\theta"]
n_params = len(labels)


# --- 3. 绘制 Corner 图 ---

# 仅绘制 Gamma 样本
fig = corner.corner(
    samples_gamma,
    labels=labels,
    color=main_color,
    fill_contours=True,    
    plot_datapoints=False, 
    show_titles=False,     
    smooth=1.0,
    range=ranges,
    quantiles=[0.16, 0.5, 0.84], # 添加中位数线
    hist_kwargs={"density": True, "color": main_color, "alpha": 0.8}
)

# --- 4. 添加中位数和误差到对角线 ---

def median_err(samples):
    """计算中位数和 16th/84th 百分位误差"""
    q16, q50, q84 = np.percentile(samples, [16, 50, 84])
    return q50, q50 - q16, q84 - q50

n_bins = 40
axes = np.array(fig.axes).reshape(n_params, n_params)

for i in range(n_params):
    ax = axes[i, i]
    
    # 计算直方图的最大高度
    counts, _ = np.histogram(samples_gamma[:, i], bins=n_bins, range=ranges[i], density=True)
    ymax_counts = counts.max() if counts.size else 0

    # 设置 Y 轴上限，为文本腾出空间
    if ymax_counts > 0:
        new_top = ymax_counts * 1.5 
        cur_ymin, cur_ymax = ax.get_ylim()
        if new_top > cur_ymax:
             ax.set_ylim(0, new_top) 

    # Gamma values - top text
    m_gamma, lo_gamma, hi_gamma = median_err(samples_gamma[:, i])
    # 关键修复：使用单个反斜杠
    text_gamma = fr"${param_names[i]} = {m_gamma:.2f}^{{+{hi_gamma:.2f}}}_{{-{lo_gamma:.2f}}}$"
    
    # Add text at the top (1.1)
    ax.text(
        0.5, 1.02, text_gamma,
        color=main_color, fontsize=13,
        ha="center", va="bottom",
        transform=ax.transAxes
    )


# ... (其他代码保持不变)
# --- 5. 保存图片 ---
output_filename = "corner_gamma_single.pdf"
fig.savefig(output_filename, dpi=300, bbox_inches="tight")
print(f"🎉 Gamma Corner 图已输出: {output_filename}")

In [ ]:
import pickle
import numpy as np
import corner
import matplotlib.pyplot as plt
from dynesty import utils

# 文件路径
path_all = "/home/lz/work/pulsar_kick_velocity/mnt/BuffaloB/lz/hy_outdir/u_hype_14897_dynesty.pickle"
path_recycled = "/home/lz/work/pulsar_kick_velocity/mnt/BuffaloB/lz/hy_outdir/u_hype_3228_dynesty.pickle"

# ============== plotting style for ApJ ==============
# ============== plotting style for ApJ ==============
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["STIX Two Text", "Times", "DejaVu Serif"],
    "mathtext.fontset": "stix",
    "axes.labelsize": 16,
    "xtick.labelsize": 13,
    "ytick.labelsize": 13,
    "axes.linewidth": 1.5,
    "xtick.major.size": 6,
    "ytick.major.size": 6,
    "xtick.minor.size": 3,
    "ytick.minor.size": 3,
})

def load_samples(path):
    """从 dynesty pickle 文件中加载并重采样等权重的样本"""
    try:
        with open(path, "rb") as f:
            results = pickle.load(f)
    except FileNotFoundError:
        print(f"错误：文件未找到 at {path}")
        # 返回一个空的/占位符样本数组以防止程序崩溃
        return np.array([[np.nan, np.nan]])

    weights = np.exp(results.logwt - results.logz[-1])
    return utils.resample_equal(results.samples, weights)

# --- 1. 加载样本 ---
samples_all = load_samples(path_all)       # blue - path_all
samples_recycled = load_samples(path_recycled) # orange - path_recycled

# 检查样本维度是否正确 (lognormal 分布应有 2 个参数: mu, sigma)
if samples_all.shape[1] != 2 or samples_recycled.shape[1] != 2:
    print("\n警告：加载的样本不是预期的 2 个参数。请检查文件内容。")
    # 如果维度不匹配，后续绘图可能会失败或结果错误
    if samples_all.shape[1] == 2 and samples_recycled.shape[1] == 2:
        pass # 继续，因为至少是 2 维
    else:
        # 如果维度确实不匹配，停止执行
        print("停止执行：样本维度错误。")
        # exit() # 实际运行时可能需要注释掉这行
        
# --- 2. 定义绘图参数 ---
# 使用专业和谐的颜色
orange_color = "#E67E22"   # 柔和橙色 (ColorBrewer OrRd_3[2]) - recycled
blue_color = "#2C7BB6"     # 深蓝色 (ColorBrewer Blues_3[2]) - all

# 定义坐标轴标签（包含单位）和参数名称（仅LaTeX命令，无$符号）
# Log-normal distribution parameters: mu, sigma (for the underlying normal distribution)
labels = [r"$\mathrm{\mu}$", r"$\mathrm{\sigma}$"]
# 关键修复：使用单个反斜杠表示LaTeX命令
param_names = [r"\mu", r"\sigma"]
# 经验范围 (可以根据实际数据调整)
ranges = [(4, 5.2), (0.7, 1.3)] 
n_params = len(labels)


# --- 3. 绘制 Corner 图 ---
'''
# First (blue - path_all)
fig = corner.corner(
    samples_all,
    labels=labels,
    color=blue_color,
    fill_contours=False,
    plot_datapoints=True,
    show_titles=False,
    smooth=1.0,
    range=ranges,
    quantiles=[0.16, 0.84],
    hist_kwargs={"density": True, "color": blue_color}
)

# Overlay second (orange - path_recycled)
corner.corner(
    samples_recycled,
    fig=fig,
    color=orange_color,
    fill_contours=False,
    plot_datapoints=True,
    show_titles=False,
    smooth=1.0,
    range=ranges,
    quantiles=[0.16, 0.84],
    alpha=0.7,
    hist_kwargs={"density": True, "color": orange_color}
)
'''
# --- 4. 添加中位数和误差到对角线 ---
# --- 3. 绘制 Corner 图 (Log-normal 比较) ---

# First (blue - path_all)
fig = corner.corner(
    samples_all,
    labels=labels,
    color=blue_color,
    fill_contours=True,      # <--- 关键修改：填充等高线
    plot_datapoints=False,   # <--- 关键修改：移除散点
    show_titles=False,
    smooth=1.0,
    range=ranges,
    quantiles=[0.16, 0.84],
    hist_kwargs={"density": True, "color": blue_color}
)

# Overlay second (orange - path_recycled)
corner.corner(
    samples_recycled,
    fig=fig,
    color=orange_color,
    fill_contours=True,     # <--- 保持 False 以便轮廓清晰可见
    plot_datapoints=False,   # <--- 关键修改：移除散点
    show_titles=False,
    smooth=1.0,
    range=ranges,
    quantiles=[0.16, 0.84],
    hist_kwargs={"density": True, "color": orange_color}
)
def median_err(samples):
    """计算中位数和 16th/84th 百分位误差"""
    q16, q50, q84 = np.percentile(samples, [16, 50, 84])
    return q50, q50 - q16, q84 - q50

n_bins = 40
axes = np.array(fig.axes).reshape(n_params, n_params)

for i in range(n_params):
    ax = axes[i, i]
    
    # 计算两个样本在相同 bins/range 下的直方值 (density)
    # n_bins 只是一个建议值，corner 的默认 binning 可能会不同
    bins = np.linspace(ranges[i][0], ranges[i][1], n_bins + 1)
    # 使用 corner 默认的 binning 策略来获取最大高度可能更准确，但为了遵循您的原代码逻辑，我们继续使用自定义 bins/density
    
    # Note: Corner.corner when called with hist_kwargs={"density": True} will use density=True for the histograms.
    counts_all, _ = np.histogram(samples_all[:, i], bins=bins, density=True)
    counts_recycled, _ = np.histogram(samples_recycled[:, i], bins=bins, density=True)
    
    # 取最大高度
    ymax_counts = max(counts_all.max() if counts_all.size else 0,
                      counts_recycled.max() if counts_recycled.size else 0)

    # 给一点顶部余量（比如 15%）
    if ymax_counts > 0:
        # 新的顶部限制设置为 ymax 的 1.5 倍或 2 倍，确保文字有足够的空间
        new_top = ymax_counts * 1.5 
        cur_ymin, cur_ymax = ax.get_ylim()
        # 仅在 new_top 大于当前 ymax 且不为 0 时设置
        if new_top > cur_ymax:
             # 设置新上限，将下限保持为 0
             ax.set_ylim(0, new_top) 

    # Blue values (samples_all) - top text
    m_blue, lo_blue, hi_blue = median_err(samples_all[:, i])
    # 关键修复：使用单个反斜杠
    text_blue = fr"${param_names[i]} = {m_blue:.2f}^{{+{hi_blue:.2f}}}_{{-{lo_blue:.2f}}}$"
    
    # Orange values (samples_recycled) - middle text
    m_orange, lo_orange, hi_orange = median_err(samples_recycled[:, i])
    text_orange = fr"${param_names[i]} = {m_orange:.2f}^{{+{hi_orange:.2f}}}_{{-{lo_orange:.2f}}}$"
    
    # Add blue text (path_all) at the top (1.1)
    ax.text(
        0.5, 1.1, text_blue,
        color=blue_color, fontsize=13,
        ha="center", va="bottom",
        transform=ax.transAxes
    )
    
    # Add orange text (path_recycled) below blue (1.0)
    ax.text(
        0.5, 1.0, text_orange,
        color=orange_color, fontsize=13,
        ha="center", va="bottom",
        transform=ax.transAxes
    )

# --- 5. 保存图片 ---
output_filename = "corner_lognormal.pdf"
fig.savefig(output_filename, dpi=300, bbox_inches="tight")
print(f"🎉 Corner 图已输出: {output_filename}")